## Parte 1: El Problema - El Texto es solo una "Sopa de Letras" para la Computadora

Cuando partimos de una lista de caracteres o de frases muy desordenadas, podemos organizarlas en forma de lista, pero aun así el sistema no obtiene significado real de ese texto. Para poder preguntarle cosas al sistema —por ejemplo, “¿quién es el sujeto?”, “¿de quién se habla?”, “¿qué acción va a realizar?” o “¿quiénes son las personas que van a viajar?”— es necesario ir más allá de la simple lista de caracteres. Con texto sucio o desordenado, el sistema no puede responder preguntas ni generar clasificaciones útiles.

Por ejemplo, en la frase “Chang, viajará a París”, la coma pegada al nombre “Chang” y la conjugación “viajará” (en lugar de “viajar”) son señales de que el texto está sucio y no normalizado. Si quiero reducir a temas, “viajará” no es el tema en sí; el tema es “viaje” o “viajar”. Por eso necesitamos construir un objeto semántico, no solo dividir en unidades mínimas (tokens), sino también llegar a las raíces de las palabras (lemmas) que nos permitan identificar o ubicar el tema. Ahi cuando necesitamos encontrar relaciones es cuando viene la gran libreria SPACY, que va a convertir la lista de caracteres en un obejto semantico, la libreria lo llama doc.


Este paso es clave porque a partir de ahí podemos:

Normalizar palabras para agrupar conceptos bajo un mismo tema.
Identificar las acciones y sus sujetos.
Reconocer entidades como personas, lugares, fechas u organizaciones.

En otras palabras, trabajar con entidades y relaciones es lo que le da sentido a los datos y permite que el sistema entienda “quién hace qué” y “de qué tema se está hablando”. Sin ese procesamiento previo, no tenemos forma de saber de antemano si estamos trabajando con entidades, ni de hacer consultas semánticas útiles.

In [1]:
texto = "La CEO de ACME, Lisa Chang, viajará a México para supervisar los nuevos proyectos de energía renovable. Ella se mostró muy entusiasmada." #Creo una variable con una cadena de caracteres, una lista de caracteres
print(texto)

La CEO de ACME, Lisa Chang, viajará a México para supervisar los nuevos proyectos de energía renovable. Ella se mostró muy entusiasmada.


¿Cómo contamos las palabras acá?
len(texto) nos da los caracteres, no sirve. Podríamos usar texto.split(). Hagamos la prueba.

In [2]:
palabras = texto.split()
print(palabras)

['La', 'CEO', 'de', 'ACME,', 'Lisa', 'Chang,', 'viajará', 'a', 'México', 'para', 'supervisar', 'los', 'nuevos', 'proyectos', 'de', 'energía', 'renovable.', 'Ella', 'se', 'mostró', 'muy', 'entusiasmada.']


 Limitaciones (El "Porqué" necesitamos algo mejor):

- "ACME," y "Chang," tienen la coma pegada. No son palabras "limpias".
- "La" y "los" son artículos. viajará es una forma conjugada del verbo "viajar". ¿Cómo reducimos todo a su forma base para poder analizar frecuencias o temas?
- ¿Cómo sabe la máquina que "ACME" es una organización y "Lisa Chang" es una persona? Para Python, ahora mismo, son solo cadenas de texto idénticas a "proyectos".

## Parte 2: La Solución - spaCy, el Intérprete Lingüístico

In [3]:
!pip install spacy -q #La q es para que no me muestre todo el procedimiento que va instalando
!python -m spacy download es_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 98.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
import spacy

In [10]:
# Cargar el modelo de español
nlp = spacy.load("es_core_news_sm") #Le cargo mi corpus

# Procesar nuestro texto
doc = nlp(texto) #voy a convertir en un doc, y le voy a pasar el modelo al texto para ver que reconoce

print(doc)

La CEO de ACME, Lisa Chang, viajará a México para supervisar los nuevos proyectos de energía renovable. Ella se mostró muy entusiasmada.


## Parte 3: Desglosando el Objeto Doc - Los Fundamentos de NLP en Acción

**Tokenización (La Unidad Mínima)**

Definición: Un token es la unidad mínima de texto con significado (una palabra, un número, un signo de puntuación).

In [6]:
for token in doc: #quiero que se me imprima cada uno de los token de ese documentos, ya ahora las comas y los puntos aparecen como un elemento significativo, a parte
    print(token.text)

La
CEO
de
ACME
,
Lisa
Chang
,
viajará
a
México
para
supervisar
los
nuevos
proyectos
de
energía
renovable
.
Ella
se
mostró
muy
entusiasmada
.


**Lematización (La Raíz de la Palabra)**

Definición: El "lema" es la forma canónica o de diccionario de una palabra. Es la clave para agrupar palabras con el mismo significado. Creo un tabla en donde obteno la palabra y la raiz, es decir, el lema que es de donde proviene la palabra. En el primer ejemplo "La" proviene de "El". Los verbos los asocia muy bien, pero tenemos problemas con el genero.

In [7]:
print("PALABRA\t\tLEMA")
print("-------\t\t----")
for token in doc:
    # Imprimimos el texto del token y su lema
    print(f"{token.text:<15}\t{token.lemma_}")

PALABRA		LEMA
-------		----
La             	el
CEO            	CEO
de             	de
ACME           	ACME
,              	,
Lisa           	Lisa
Chang          	Chang
,              	,
viajará        	viajar
a              	a
México         	México
para           	para
supervisar     	supervisar
los            	el
nuevos         	nuevo
proyectos      	proyecto
de             	de
energía        	energía
renovable      	renovable
.              	.
Ella           	él
se             	él
mostró         	mostrar
muy            	mucho
entusiasmada   	entusiasmado
.              	.


**Part-of-Speech (POS) Tagging (El Rol Gramatical)**

Definición: Etiquetar cada palabra según su función gramatical (Sustantivo, Verbo, Adjetivo, etc.). Despues de esto si voy a poder empezar a operar para obtener algunos insigns de textos mas grande.

In [8]:
print("PALABRA\t\tPOS\t\tEXPLICACIÓN")
print("-------\t\t---\t\t-----------")
for token in doc:
    print(f"{token.text:<15}\t{token.pos_:<10}\t{spacy.explain(token.pos_)}")

PALABRA		POS		EXPLICACIÓN
-------		---		-----------
La             	DET       	determiner
CEO            	PROPN     	proper noun
de             	ADP       	adposition
ACME           	PROPN     	proper noun
,              	PUNCT     	punctuation
Lisa           	PROPN     	proper noun
Chang          	PROPN     	proper noun
,              	PUNCT     	punctuation
viajará        	VERB      	verb
a              	ADP       	adposition
México         	PROPN     	proper noun
para           	ADP       	adposition
supervisar     	VERB      	verb
los            	DET       	determiner
nuevos         	ADJ       	adjective
proyectos      	NOUN      	noun
de             	ADP       	adposition
energía        	NOUN      	noun
renovable      	ADJ       	adjective
.              	PUNCT     	punctuation
Ella           	PRON      	pronoun
se             	PRON      	pronoun
mostró         	VERB      	verb
muy            	ADV       	adverb
entusiasmada   	ADJ       	adjective
.              	PUNCT     	punctu

**Conclusion**: Spicy me va a permitir convertir una lista de caracteres que la maquina no puede entender, lo va a convertir en un documento con el cual podrè realizar operaciones semanticas, me va a reconocer la estructura, entidades y lo mas importante va a hacer la operacion de tokenizar.